In [1]:
import sys
parentdir = ".."
sys.path.insert(0, parentdir)

import requests
from os import path
from tqdm.notebook import tqdm
import pandas as pd
import time

from metrics.WebResource import WebResource

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 100.0.4896
[WDM] - Get LATEST driver version for 100.0.4896
[WDM] - Driver [/Users/gaignard-a/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


# 1. Retrieve a list of target URLs

In [9]:
target_urls = ["http://bio.tools/bwa", "http://bio.tools/jaspar", "https://doi.pangaea.de/10.1594/PANGAEA.942997", "https://zenodo.org/record/6411574#.Ykw0bi8itqs"]

In [10]:
#for url in tqdm(target_urls):
#    print(url)

# 2. Feed the RDF graph

In [11]:
from rdflib import ConjunctiveGraph, Namespace, URIRef
from rdflib.namespace import RDF, RDFS

schema = Namespace("http://schema.org/")

In [12]:
KG = ConjunctiveGraph()

In [13]:
for u in tqdm(target_urls):
    wr = WebResource(url=u)
    KG += wr.get_rdf()
    print(f"Loaded {len(KG)} triples")

print(f"Loaded {len(KG)} triples")

  0%|          | 0/4 [00:00<?, ?it/s]

Loaded 124 triples
Loaded 219 triples
http://schema.org/
Loaded 380 triples


https://schema.org/
Loaded 440 triples
Loaded 440 triples


In [14]:
KG.serialize("scrapped_bioschemas.ttl", format="turtle")

<Graph identifier=N50cfbbd444fe4d2999d0ec02e19be206 (<class 'rdflib.graph.ConjunctiveGraph'>)>

In [15]:
!cat scrapped_bioschemas.ttl

@prefix ns1: <http://schema.org/> .
@prefix ns2: <http://bioschemas.org/> .
@prefix ns3: <http://ogp.me/ns#> .
@prefix ns4: <https://bio.tools/ontology/> .
@prefix ns5: <http://purl.org/dc/terms/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<file:///Users/gaignard-a/Documents/Dev/fair-checker/notebooks/> ns3:description "This dataset contains impact metrics and indicators for a set of publications that are related to the COVID-19 infectious disease and the coronavirus that causes it. It is based on: Τhe CORD-19 dataset released by the team of Semantic Scholar1 and Τhe curated data provided by the LitCovid hub2. These data have been cleaned and integrated with data from COVID-19-TweetIDs and from other sources (e.g., PMC). The result was dataset of 503,979 unique articles along with relevant metadata (e.g., the underlying citation network). We utilized this dataset to produce, for each article, the values of the following impact measures: Influence: Citation-based measure refl

# 3. Display Bioschemas classes and properties counts.

In [36]:
def get_dataframe_from_query_results(res):    
    return pd.DataFrame(res.bindings)

### display used classes
classes_counts = """
SELECT ?c (count(?c) as ?count) WHERE {
    ?s rdf:type ?c .
} 
GROUP BY ?c
ORDER BY DESC(?count)
"""

res = KG.query(classes_counts)
print()
print("Used classes")
df = get_dataframe_from_query_results(res)
df


Used classes


,c,count
0,http://bioschemas.org/types/FormalParameter,22
1,http://schema.org/Person,16
2,http://schema.org/CreativeWork,15
3,http://schema.org/DataDownload,7
4,http://schema.org/Dataset,4
5,http://schema.org/SoftwareApplication,2
6,schema:Person,2
7,http://schema.org/QuantitativeValue,2
8,http://schema.org/Place,2
9,http://schema.org/CreativeWorkSeries,2


In [37]:
### display used properties
property_counts = """
SELECT ?p (count(?p) as ?count) WHERE {
    ?s ?p ?o .
} 
GROUP BY ?p
ORDER BY DESC(?count)
"""

res = KG.query(property_counts)
print()
print("Used properties")
df = get_dataframe_from_query_results(res)
df


Used properties


,count,p
0,82,http://www.w3.org/1999/02/22-rdf-syntax-ns#type
1,56,http://schema.org/name
2,37,http://schema.org/identifier
3,34,http://schema.org/citation
4,30,http://schema.org/sameAs
5,22,http://bioschemas.org/input
6,19,http://schema.org/creator
7,12,http://schema.org/url
8,11,http://schema.org/givenName
9,11,http://schema.org/familyName


# Visualization
Have a look to Seaborn and Bokeh libraries
https://seaborn.pydata.org
https://bokeh.org